In [17]:
import re
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import numpy as np
plt.rcParams['animation.ffmpeg_path'] = '/Users/chris/Downloads/ffmpeg'
c1 = mpl.colormaps['viridis']
c2 = mpl.colormaps['cool']

lines = open('./data.txt', 'r').read().splitlines()

class Point:
    def __init__(self, loc):
        (x, y) = loc
        self.x = int(x)
        self.y = int(y)
    
    def dist(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)

class Sensor:
    def __init__(self, sens_loc: Point, beac_loc: Point):
        self.sens_loc = sens_loc
        self.beac_loc = beac_loc
        self.radius = sens_loc.dist(beac_loc)
        self.min_x = sens_loc.x - self.radius
        self.max_x = sens_loc.x + self.radius
        self.min_y = sens_loc.y - self.radius
        self.max_y = sens_loc.y + self.radius
        self.width = self.max_x - self.min_x
        self.height = self.max_y - self.min_y

    def test_point(self, p: Point):
        return self.sens_loc.dist(p) <= self.radius
    
    def bounds_at_y(self, y: int):
        if abs(y - self.sens_loc.y) > self.radius: return None
        x_val = self.radius - abs(y - self.sens_loc.y)
        return (self.sens_loc.x - x_val, self.sens_loc.x + x_val)

location_re = re.compile(r'x=(-?\d+), y=(-?\d+)')

lines = [line.split(': ') for line in lines]
items = [[Point(location_re.search(val).groups()) for val in line] for line in lines]
items = [Sensor(i[0], i[1]) for i in items]

mins = (min([p.min_x for p in items]), min([p.min_y for p in items]), min([p.radius for p in items]))
maxs = (max([p.max_x for p in items]), max([p.max_y for p in items]), max([p.radius for p in items]))

bounds = ((mins[0], mins[1]), (maxs[0], maxs[1]))

width = bounds[1][0] - bounds[0][0]
height = bounds[1][1] - bounds[0][1]

In [ ]:
test_y = 2000000
count = 0
for i in range(bounds[0][0], bounds[1][0] + 1):
    p = Point((i, test_y))
    count += any(map(lambda s: s.test_point(p), items))

count

In [24]:
from functools import reduce

n_b = ((0, 4000000), (0, 4000000))

def combine_ranges(ranges):
    def combine_2(r1, r2):
        if r1[0] <= r2[1] and r2[0] <= r1[1]:
            return (min(r1[0], r2[0]), max(r1[1], r2[1]))
        else:
            return None
    
    out = []
    prev = ranges[0]
    for i in range(1, len(ranges)):
        cur = ranges[i]
        res = combine_2(prev, cur)
        if res:
            prev = res
        else:
            out.append(prev)
            prev = cur
    out.append(prev)
    out[0] = (max(0, out[0][0]), out[0][1])
    out[-1] = (out[-1][0], min(4000000, out[-1][1]))
    return out

rows = []
for y in range(n_b[1][0], n_b[1][1] + 1):
    ranges = [s.bounds_at_y(y) for s in items]
    ranges = [b for b in ranges if b]
    ranges = sorted(ranges, key=lambda x: x[0])
    row = combine_ranges(ranges)
    if len(row) > 1:
        rows.append((y, row))

rows
# ranges

[(3186981, [(0, 3334478), (3334480, 4000000)])]

In [26]:
rows[0][0] + (rows[0][1][0][1] + 1) * 4000000

13337919186981

In [22]:
smallest = min([item.width for item in items])
smallest = next(x for x in items if x.width == smallest)

parts = smallest.height // 10
ys = [smallest.min_y + (i * parts) for i in range(10)] + [smallest.max_y]

print(smallest.min_y, smallest.max_y, smallest.height)

[(y, smallest.bounds_at_y(y)) for y in ys]

# for i in range(item.min_y, item.max_y + 1):
#     r = item.bounds_at_y(i)
#     row = ss[i - item.min_y]
#     for x in range(r[0], r[1] + 1):
#         row[x] = 1

# item.width, item.height

1670478 2282136 611658


[(1670478, (3011731, 3011731)),
 (1731643, (2950566, 3072896)),
 (1792808, (2889401, 3134061)),
 (1853973, (2828236, 3195226)),
 (1915138, (2767071, 3256391)),
 (1976303, (2705906, 3317556)),
 (2037468, (2767063, 3256399)),
 (2098633, (2828228, 3195234)),
 (2159798, (2889393, 3134069)),
 (2220963, (2950558, 3072904)),
 (2282136, (3011731, 3011731))]